<a href="https://colab.research.google.com/github/Kim-Yeon-Jun/Machine_Learning/blob/main/Sigp4_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#인공신경망 구현하기 with Pytorch

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris

from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


In [ ]:
iris = load_iris()
df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
df['label'] =  iris.target

In [ ]:
y = df['label']
X = df.drop(['label'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify = y)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 10, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle = True)

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.input_layer = nn.Linear(4, 16)
    self.hidden_layer1 = nn.Linear(16,32)
    self.output_layer = nn.Linear(32, 3)
    self.relu = nn.ReLU()

  def forward(self,x):
    out = self.relu(self.input_layer(x))
    out = self.relu(self.hidden_layer1(out))
    out = self.output_layer(out)
    return out

In [ ]:
device = 'cuda' if  torch.cuda.is_available() else 'cpu'
print(f'device = {device}')

model = NeuralNetwork().to(device)

device = cuda


In [ ]:
#모델 컴파일
learning_rate = 0.001
loss =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [ ]:
def train_loop(train_loader, model, loss_fn, optimizer):
  size = len(train_loader.dataset)
  for batch, (X,y) in enumerate(train_loader):
    X, y = X.to(device), y.to(device)
    pred = model(X)

    #손실 계산
    loss = loss_fn(pred, y)

    #역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss, current = loss.item(), batch * len(X)
    print(f'loss : {loss:>7f} [{current:>5d}]/{size:5d}')

In [ ]:
def test_loop(test_loader, model, loss_fn):
  size = len(test_loader.dataset)
  num_batches = len(test_loader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in test_loader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error : \n Accuracy : {(100*correct):0.1f}%, Avg loss : {test_loss:8f}\n")

In [ ]:
epochs = 10

for i in range(epochs):
  print(f"Epoch {i+1} \n---------------")
  train_loop(train_dataloader, model, loss, optimizer)
  test_loop(test_dataloader, model, loss)
print("Done!")

Epoch 1 
---------------
loss : 1.061184 [    0]/  112
loss : 1.065614 [   10]/  112
loss : 0.986468 [   20]/  112
loss : 1.010825 [   30]/  112
loss : 1.055754 [   40]/  112
loss : 1.004410 [   50]/  112
loss : 0.977798 [   60]/  112
loss : 0.965269 [   70]/  112
loss : 1.066278 [   80]/  112
loss : 0.961919 [   90]/  112
loss : 0.963350 [  100]/  112
loss : 0.926853 [   22]/  112
Test Error : 
 Accuracy : 60.5%, Avg loss : 0.972791

Epoch 2 
---------------
loss : 0.974821 [    0]/  112
loss : 0.957860 [   10]/  112
loss : 0.968445 [   20]/  112
loss : 0.991759 [   30]/  112
loss : 0.964476 [   40]/  112
loss : 0.908039 [   50]/  112
loss : 0.980181 [   60]/  112
loss : 0.870738 [   70]/  112
loss : 0.936861 [   80]/  112
loss : 0.869548 [   90]/  112
loss : 0.871144 [  100]/  112
loss : 1.028698 [   22]/  112
Test Error : 
 Accuracy : 76.3%, Avg loss : 0.905203

Epoch 3 
---------------
loss : 0.887674 [    0]/  112
loss : 0.822056 [   10]/  112
loss : 0.866697 [   20]/  112
loss : 